In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:

##? 1st - left, 2nd - right, 3rd - up, 4th - down
##? (x, y) - order

SquareVersors = np.array([[-1, 0], [1, 0], [0, 1], [0, -1]], dtype=np.int32)

##! F points

arrFC = np.array([1, 0, 0, 0], dtype=np.int32)
arrFB = np.array([0, 1, 0, 0], dtype=np.int32)
arrFE = np.array([0, 0, 0, 1], dtype=np.int32)
arrFD = np.array([0, 0, 1, 0], dtype=np.int32)


##! B points

arrBC = np.array([1, 0, 1, 1], dtype=np.int32)
arrBB = np.array([0, 1, 1, 1], dtype=np.int32)
arrBE = np.array([1, 1, 0, 1], dtype=np.int32)
arrBD = np.array([1, 1, 1, 0], dtype=np.int32)


##! V points

arrVC = np.array([1, 0, 0, 1], dtype=np.int32)
arrVB = np.array([0, 1, 1, 0], dtype=np.int32)
arrVE = np.array([0, 1, 0, 1], dtype=np.int32)
arrVD = np.array([1, 0, 1, 0], dtype=np.int32)


###! L points

arrLH = np.array([0, 0, 1, 1], dtype=np.int32)
arrLV = np.array([1, 1, 0, 0], dtype=np.int32)


In [3]:
dictStr2Arr = { 
    'FC': arrFC, 'FB': arrFB, 'FE': arrFE, 'FD': arrFD,
    'BC': arrBC, 'BB': arrBB, 'BE': arrBE, 'BD': arrBD,
    'VC': arrVC, 'VB': arrVB, 'VE': arrVE, 'VD': arrVD,
    'LH': arrLH, 'LV': arrLV
}

lookUpFunc = lambda x: dictStr2Arr.get(x, x)  # Default to x if not found in dict

In [4]:

##! improve vec comoutation: outside the loop

def getPointsSquare(x, y, arr):
    pos0 = np.array([2*x+1, 2*y+1], dtype=np.int32)
    vec = arr.reshape(-1, 1)*SquareVersors
    
    mask = np.all(vec == 0, axis=1)
    result = np.full_like(vec, -1)
    result[~mask] = pos0 + vec[~mask]

    return result


vecTot = np.array([1, 1, 1, 1], dtype=np.int32)

def getAllPointsSquare(x, y,):
    pos0 = np.array([2*x+1, 2*y+1], dtype=np.int32)
    vec = vecTot.reshape(-1, 1)*SquareVersors
    
    result = vec + pos0
    return result

In [5]:
def goalTest(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    
    dotsX2 = np.zeros((mSize-1, mSize, 2, 2), dtype=np.int32) ##! slices along x constant
    dotsY2 = np.zeros((mSize, mSize-1, 2, 2), dtype=np.int32) ##! slices along y constant
    
    dotsX2[:, :, 0, :] = dataPointsGrid[:-1, :, 1]
    dotsX2[:, :, 1, :] = dataPointsGrid[1:, :, 0]

    dotsY2[:, :, 0, :] = dataPointsGrid[:, :-1, 2]
    dotsY2[:, :, 1, :] = dataPointsGrid[:, 1:, 3]
    
    matLogicalX = (dotsX2[:, :, 0, :] == dotsX2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 0, :] > -1))
    matLogicalY = (dotsY2[:, :, 0, :] == dotsY2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 1, :] > -1))
    
    resLogicalX = np.all(matLogicalX, axis=2)
    resLogicalY = np.all(matLogicalY, axis=2)
    
    return resLogicalX, resLogicalY
    # return np.all(resLogicalX) and np.all(resLogicalY)

In [6]:
def strategy1(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    matLog1 = np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3))
    idxMat = np.where(matLog1)
     
    
    def strategy1Aux(tableArr, x, y):
    
        result = getPointsSquare(x, y, tableArr)
        bool2 = np.any(np.logical_or(result == 0, result == 2*mSize))
        
        return bool2
    
    
    for x, y in zip(*idxMat):
        
        setF = ['FC', 'FB', 'FE', 'FD']
        setB = ['BC', 'BB', 'BE', 'BD']
        setV = ['VC', 'VB', 'VE', 'VD']
        setL = ['LH', 'LV']
        
        stringValue = tableInit[x, y]
        
        # print(stringValue)

        if stringValue in setF:
            bool1 = True
            while bool1:
                
                string2 = setF[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                
                setF.remove(string2)
                
            tableInit[x, y] = string2
            
        elif stringValue in setB:
            
            setB.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setB[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setB.remove(string2)
                
            tableInit[x, y] = string2

        elif stringValue in setV:
            
            setV.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setV[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setV.remove(string2)
                
            tableInit[x, y] = string2
                
        elif stringValue in setL:
            
            setL.remove(stringValue)
            bool1 = True
            while bool1:
                
                string2 = setL[0]
                tableArr[x, y] = lookUpFunc(string2)
                bool1 = strategy1Aux(tableArr[x, y], x, y)
                setL.remove(string2)
                
            tableInit[x, y] = string2

In [50]:
def strategyZero(tableInit):

    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=np.int32)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=np.int32)
    matLog1 = np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3))
    
    setF = np.array(['FC', 'FB', 'FE', 'FD'])
    setB = np.array(['BC', 'BB', 'BE', 'BD'])
    setV = np.array(['VC', 'VB', 'VE', 'VD'])
    setL = np.array(['LH', 'LV'])
    
    idxMatB = np.isin(tableInit, setB)
    idxMatL = np.isin(tableInit, setL)
    
    ##! avoid the V pices for now: focused on the B and L pieces

    ##! obtain the external B and L pieces
    idxMatAux = np.logical_or(idxMatB, idxMatL)
    pointsDeterm = np.where(np.logical_and(idxMatAux, matLog1))
    
    idxDeterm = np.array(list(zip(*pointsDeterm)))
    idGridDeterm = np.array([tableInit[x, y] for x, y in zip(*idxDeterm)])
    
    ##! putting the deterministic pices in the proper place
    for i in range(idxDeterm.shape[0]):
        x, y = idxDeterm[i]
        idPoint = tableInit[x, y]
        
        if idPoint in setB:
            
            gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setB], dtype=np.int32)
            checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
            
            idxCheckBoundaries = np.where(~checkBoundaries)[0]
            
            if len(idxCheckBoundaries) > 1:
                print("error: more than one possible solution")
            else:
                tableInit[x,y] = setB[idxCheckBoundaries[0]]
                tableArr[x, y] = lookUpFunc(setB[idxCheckBoundaries[0]])
            
        elif idPoint in setL:
            
            gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setL], dtype=np.int32)
            checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
            
            idxCheckBoundaries = np.where(~checkBoundaries)[0]
            
            if len(idxCheckBoundaries) > 1:
                print("error: more than one possible solution")
            else:
                tableInit[x,y] = setL[idxCheckBoundaries[0]]
                tableArr[x, y] = lookUpFunc(setL[idxCheckBoundaries[0]])
    
    
    # print(tableInit)
    
    ##! building the rest of the external layer which now is deterministic
    
    ##! Auxiliar Important Functions
    def getAdjacentPoints(x, y, maskFrameCheck):
        
        checkFrame2 = np.full_like(tableInit, False)
        xMod = x-1
        yMod = y-1
        if xMod < 0:
            xMod = 0
        if yMod < 0:
            yMod = 0
        checkFrame2[xMod:x+2, y] = True
        checkFrame2[x, yMod:y+2] = True
        
        checkFrame2 = np.logical_and(checkFrame2, maskFrameCheck)
        # print(checkFrame2)
        return np.where(checkFrame2)
    
    
    def strategyOne(x2, y2, maskFrameCheck):
        
        idx2 = getAdjacentPoints(x2, y2, maskFrameCheck)
        detAdjPoints = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idx2)], dtype=np.int32)
        
        setF = np.array(['FC', 'FB', 'FE', 'FD'])
        setB = np.array(['BC', 'BB', 'BE', 'BD'])
        setV = np.array(['VC', 'VB', 'VE', 'VD'])
        setL = np.array(['LH', 'LV'])
        
        idPoint = tableInit[x2, y2]
        
        print(idPoint)
        print([tableInit[x, y] for x, y in zip(*idx2)])
        
        ## get the points of the boundary
        pointsXY = getAllPointsSquare(x2, y2)
        
        ## get the points and the coordinates of the deterministic external points
        matches = np.array([np.any((detAdjPointsAux[:, np.newaxis, :] == pointsXY).all(axis=2), axis=1) for detAdjPointsAux in detAdjPoints])
        idxMatch = np.where(matches)
        idxMatchMat = np.array(list(zip(*idxMatch)))
        
        print(detAdjPoints)
        
        ## get the proper points on the boundary
        if idxMatchMat.size != 0:
            properPointsBoundary = detAdjPoints[idxMatchMat[:, 0], idxMatchMat[:, 1]]
        else:
            properPointsBoundary = np.array([], dtype=np.int32)
        
        actions = []
        
        if idPoint in setF:
            
            if properPointsBoundary.size == 0:
                return setF
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setF], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setF[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setV:
            
            if properPointsBoundary.size == 0:
                return setV
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setV], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setV[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setL:
            
            if properPointsBoundary.size == 0:
                return setL
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setL], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setL[*np.where(similarityVec == np.max(similarityVec))]
            
        elif idPoint in setB:
            
            ## in case of not having points on the boundary, the full set is returned
            if properPointsBoundary.size == 0:
                return setB
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setB], dtype=np.int32)

            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=np.int32)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setB[*np.where(similarityVec == np.max(similarityVec))]
    
        return actions
    
    
    maskBorder = np.full_like(tableInit, False)
    maskBorder[0, :] = True
    maskBorder[-1, :] = True
    maskBorder[:, 0] = True
    maskBorder[:, -1] = True
    
    # print(maskBorder)
    
    maskFrameCheck = np.full_like(tableInit, False)
    maskAux = np.full_like(tableInit, False)
    
    for x, y in zip(*idxDeterm):
        maskFrameCheck[x, y] = True
            
        maskAux[x-1:x+2, y] = True
        maskAux[x, y-1:y+2] = True
        maskAux[x, y] = False
        
    maskBorderZero = np.logical_and(maskBorder, maskAux)
    idx2ChangeBorderZero = np.where(maskBorderZero)
    
    # print(maskFrameCheck)
    print(maskBorderZero)
    
    idx2ChangeBorderZeroVec = np.array(list(zip(*idx2ChangeBorderZero)))
    
    for i in range(1):
        x2, y2 = idx2ChangeBorderZeroVec[i]
        # print(x2, y2)
        # print(tableInit[x2, y2])
        actions = strategyOne(x2, y2, maskBorderZero)
        print(actions)
        print()
    
    
    
    # def strategyRecurrentZero(idx2ChangeBorder, idxDeterm, bool1=False):
        
    #     detPointsGrid = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idxDeterm)], dtype=np.int32)
    #     unknownPoints = []
        
    #     for x, y in zip(*idx2ChangeBorder):
            
    #         idPoint = tableInit[x, y]
            
    #         setF = ['FC', 'FB', 'FE', 'FD']
    #         setB = ['BC', 'BB', 'BE', 'BD']
    #         setV = ['VC', 'VB', 'VE', 'VD']
    #         setL = ['LH', 'LV']
            
    #         if idPoint in setF:
    #             while True:
    #                 # print(idPoint, setF)
    #                 compatNum = strategyZeroAux(tableArr[x,y], detPointsGrid, x, y, bool1)
    #                 # print(compatNum)
    #                 if compatNum == 1:
    #                     idPoint = setF[0]
    #                     break
                    
    #                 setF.remove(idPoint)
    #                 if len(setF) < 1:
    #                     break
                    
    #                 idPoint = setF[0]
    #                 tableArr[x, y] = lookUpFunc(idPoint)
                    
    #         elif idPoint in setV:
    #             #? I am not sure if it englobes all the cases
    #             while True:
    #                 compatNum = strategyZeroAux(tableArr[x,y], detPointsGrid, x, y, bool1)
                                        
    #                 if compatNum != 0:
    #                     idPoint = setV[0]
    #                     break
                        
    #                 setV.remove(idPoint)
    #                 if len(setV) < 1:
    #                     break
                    
    #                 idPoint = setV[0]
    #                 tableArr[x, y] = lookUpFunc(idPoint)
    #         else:
    #             pass
            
    #         tableInit[x, y] = idPoint
    #         maskFrameCheck[x, y] = True
            
            
    #         if len(setF) == 0:
    #             unknownPoints.append([x, y])
    #         elif len(setV) == 0:
    #             unknownPoints.append([x, y])
    #         else:
    #             pass
            
    #     return maskFrameCheck, np.array(unknownPoints, dtype=np.int32)

    # ##! External Layer
    # ##? Zero Term
    # maskFrameCheck = np.full_like(tableInit, False)
    # maskAux = np.full_like(tableInit, False)
        
    # for x, y in zip(*idxDeterm):
    #     maskFrameCheck[x, y] = True
            
    #     maskAux[x-1:x+2, y] = True
    #     maskAux[x, y-1:y+2] = True
    #     maskAux[x, y] = False
        
    # theMaskBorder = np.logical_and(maskBorder, maskAux)
    # idx2ChangeBorder = np.where(theMaskBorder)
    
    # maskFrameCheck, unknownZero = strategyRecurrentZero(idx2ChangeBorder, idxDeterm)
    
    # def strategyTheLoop(maskFrameCheck, unknownZero, bool1=False):
        
    #     maskFrameCheckAux = np.full_like(tableInit, False)
        
    #     for x, y in zip(*np.where(maskFrameCheck)):
    #         if unknownZero.size != 0:
    #             if np.all(np.isin(unknownZero[:,0], x) & np.isin(unknownZero[:,1], y)):
    #                 continue
    #         xMod = x-1
    #         yMod = y-1
    #         if xMod < 0:
    #             xMod = 0
    #         if yMod < 0:
    #             yMod = 0
    #         maskFrameCheckAux[xMod:x+2, y] = True
    #         maskFrameCheckAux[x, yMod:y+2] = True
        
        
    #     maskFrameCheckAux = np.logical_and(maskFrameCheckAux, maskBorder)
    #     newMaskFrameCheck =  np.logical_and(np.logical_or(maskFrameCheckAux, maskFrameCheck), np.logical_not(np.logical_and(maskFrameCheck, maskFrameCheckAux)))
        
    #     idx2ChangeBorder = np.where(newMaskFrameCheck)
    #     newMaskDeterm = np.full_like(tableInit, False)
    #     for x, y in zip(*idx2ChangeBorder):
    #         xMod = x-1
    #         yMod = y-1
    #         if xMod < 0:
    #             xMod = 0
    #         if yMod < 0:
    #             yMod = 0
    
    #         newMaskDeterm[xMod:x+2, y] = True
    #         newMaskDeterm[x, yMod:y+2] = True

    #     newMaskDeterm = np.logical_and(newMaskDeterm, maskFrameCheck)

    #     maskFrameCheckAuxNew, unknownPointsNew = strategyRecurrentZero(idx2ChangeBorder, np.where(newMaskDeterm), bool1)
        
    #     return maskFrameCheckAuxNew, unknownPointsNew
    
    # ##? Remaining Terms
    # unknownList = []
    
    # if unknownZero.size != 0:
    #     unknownList.append(unknownZero)
    
    # boolMat = True
    # counter = 0
    # unknownPointsAux = unknownZero
    
    # while boolMat:
    #     maskFrameCheck, unknownPointsAux = strategyTheLoop(maskFrameCheck, unknownPointsAux)
        
    #     if unknownPointsAux.size != 0:
    #         unknownList.append(*unknownPointsAux)
        
    #     if np.all(maskFrameCheck[0, :] & maskFrameCheck[-1, :] & maskFrameCheck[:, 0] & maskFrameCheck[:, -1]):
    #         boolMat = False
    
    # unknownList = np.array(unknownList, dtype=np.int32)
    # maskFrameCheck[unknownList[:,0], unknownList[:,1]] = False
    
    # print(maskFrameCheck)
    
    ##! Upper Layer
    
    # maskFrameCheck2 = np.full_like(tableInit, False)
    # maskFrameCheck2[1:-1, 1:-1] = True
    
    # x2, y2 = 2, 2
    # actions = strategyOneAux(x2, y2)
    # return actions


tableInit0 = np.array(  
    [['FC','BB','BC','FB'],
    ['FC','BD','FD','FD'],
    ['VD','BC','BC','BD'],
    ['FB','FB','VE','FD']] , dtype=object)

table4 = tableInit0.copy()

actions = strategyZero(table4)
# print(actions)
# print(table4)


[[False True False True]
 [False False False True]
 [False False False False]
 [False False False True]]
BB
['BB']
[[[-1 -1]
  [ 2  3]
  [ 1  4]
  [ 1  2]]]
['BB']



In [ ]:
# tableInit = np.array(
#     [['FB', 'VD', 'VD'] \
#     ,['BC', 'BB', 'LV'] \
#     ,['FB', 'FB', 'FB']] , dtype=object)


# tableInit2 = np.array(
#     [['VB', 'VD'], \
#      ['FC', 'FD']], dtype=object)


# table2 = tableInit.copy()
# table3 = tableInit2.copy()

In [ ]:
# table2 = tableInit.copy()

# setF = ['FC', 'FB', 'FE', 'FD']
# setB = ['BC', 'BB', 'BE', 'BD']
# setV = ['VC', 'VB', 'VE', 'VD']
# setL = ['LH', 'LV']


# tableArr = np.array([[lookUpFunc(x) for x in row] for row in table2], dtype=np.int32)


# # print(setB)
# # stringValue = table2[x, y]
# # setB.remove(stringValue)
# # print(setB)




# x = 0
# y = 2


# stringValue = table2[x, y]

# if stringValue in setF:
#     bool1 = True
#     while bool1:
        
#         string2 = setF[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
        
#         setF.remove(string2)
        
#     table2[x, y] = string2
    
# elif stringValue in setB:
    
#     setB.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setB[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setB.remove(string2)
        
#     table2[x, y] = string2

# elif stringValue in setV:
    
#     setV.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setV[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setV.remove(string2)
        
#     table2[x, y] = string2
        
# elif stringValue in setL:
    
#     setL.remove(stringValue)
#     bool1 = True
#     while bool1:
        
#         string2 = setL[0]
#         tableArr[x, y] = lookUpFunc(string2)
#         bool1 = strategy1(tableArr[x, y], x, y)
#         setL.remove(string2)
        
#     table2[x, y] = string2
        


# print(table2[x, y])

# while matLog2:
    
#     print(setB)
#     stringValue = table2[x, y]
#     setB.remove(stringValue)
    
#     table2[x, y] = setB[0]

#     tableArr[x, y] = lookUpFunc(setB[0])
#     result = getPointsSquare(x, y, tableArr[x, y])
#     matLog2 = np.any(np.logical_or(result == 0, result == 2*mSize))
    
    
# print(table2[x, y])